In [1]:
import pandas as pd
import numpy as np
import requests
import dill
import re
from StringIO import StringIO
from csv_pkl_sql import save_it

# Get airport information from [fallingrain.com](http://www.fallingrain.com/world/index.html)

Import cleaned location data.

In [2]:
with open('../pkl/00_cleaned_city_names.pkl', 'r') as fh:
    location_key = dill.load(fh)

In [3]:
location_key.head(2)

,location,location_type,country,province,county,city
0,Argentina-Buenos_Aires,province,Argentina,Buenos Aires,None,None
1,Argentina-CABA,province,Argentina,Ciudad de Buenos Aires,None,None


In [4]:
data_countries = location_key.country.unique()
data_countries

array(['Argentina', 'Brazil', 'Colombia', 'Dominican Republic', 'Ecuador',
       'El Salvador', 'Guatemala', 'Haiti', 'Mexico', 'Nicaragua',
       'Panama', 'United States'], dtype=object)

Scrape a table of country abbreviations from fallingrain.com that will be used to build the URLs for country information.

In [5]:
tables = pd.read_html(requests.get('http://www.fallingrain.com/world/index.html').text)
countries = pd.DataFrame({'full':tables[0].values.ravel()}).dropna()
countries[['abbrev', 'name']] = (countries.full
                                 .str.extract(r"""([A-Z]{2}) (.+)""", expand=True)
                                 )

mask = countries.name.isin(data_countries)
countries = countries[mask].reset_index(drop=True)
assert mask.sum() == len(data_countries)

countries['url'] = countries.abbrev.apply(lambda x: 'http://www.fallingrain.com/world/{}/'.format(x))

United States state abbreviations

In [6]:
state_abbreviations = pd.read_csv(StringIO(requests.get('http://www.fonz.net/blog/wp-content/uploads/2008/04/states.csv').text))

## Scrape the airports

### All countries except the United States

In [7]:
countries['airports'] = countries.loc[countries.abbrev!='US','url'].apply(lambda x: x+'airports.html')

In [8]:
df_list = list()
for url in countries.airports:
    if url is not np.NaN:
        table = pd.read_html(url)[0]
        table.columns = table.iloc[0]
        table = table.iloc[1:]
        
        country_code = re.search(r"""world\/([A-Z]{2})\/""",url).group(1)
        country_name = countries.loc[countries.abbrev==country_code, 'name'].values[0]
        table['country'] = country_name
        df_list.append(table)
        
        
airports_df_1 = pd.concat(df_list, axis=0)

### Just the United States

In [9]:
states = pd.DataFrame({'states':location_key.loc[location_key.country=='United States','province'].unique()})
states = pd.merge(states, state_abbreviations, left_on='states', right_on='State', how='left')

states.loc[states.states=='Puerto Rico', 'Abbreviation'] = 'PR'
states.loc[states.states=='American Samoa', 'Abbreviation'] = 'AS'
states.loc[states.states=='Virgin Islands', 'Abbreviation'] = 'VI'
states.drop(['State'], axis=1, inplace=True)

In [10]:
states['airports'] = states.Abbreviation.apply(lambda x: 'http://www.fallingrain.com/world/US/{}/airports.html'.format(x))

In [11]:
df_list = list()
for url in states.drop_duplicates(subset=['Abbreviation']).airports:
    if url is not np.NaN:
        table = pd.read_html(url)[0]
        table.columns = table.iloc[0]
        table = table.iloc[1:]
        
        state_code = re.search(r"""world\/US\/([A-Z]{2})\/""",url).group(1)
        state_name = states.loc[states.Abbreviation==state_code, 'states'].values[0]
        table['state'] = state_name
        table['country'] = 'United States'
        df_list.append(table)
        
airports_df_2 = pd.concat(df_list, axis=0)

## Combine airport dataframes
Combine and clean columns.

In [14]:
airports = pd.concat([airports_df_1, airports_df_2], axis=0).reset_index(drop=True)

# Clean up column names
name_mapper = dict([(x,x.lower().replace(' ','_')) 
               for x in ['Kind','City','Name','Latitude','Longitude','Max Runway']])

airports = airports.rename(columns=name_mapper)

# Column formatting
airports['latitude'] = airports.latitude.str.replace(r"""\([NS]\)""", '').astype(float)
airports['longitude'] = airports.longitude.str.replace(r"""\([EW]\)""", '').astype(float)
airports['max_runway'] = airports.max_runway.str.replace(r""" ft""", '').astype(float)

airports.head()

,city,FAA,IATA,ICAO,kind,latitude,longitude,max_runway,name,country,state
0,NaN,NaN,NaN,NaN,Small,-38.083,-70.668,5249.0,LONCOPUE,Argentina,NaN
1,NaN,NaN,NaN,NaN,Small,-38.539,-70.337,5249.0,LAS LAJAS,Argentina,NaN
2,NaN,NaN,NaN,NaN,Small,-49.995,-68.953,3960.0,COMANDANTE LUIS PIEDRABUENA,Argentina,NaN
3,NaN,NaN,NaN,NaN,Small,-38.414,-68.737,3117.0,LOMA LA LATA,Argentina,NaN
4,NaN,NaN,NaN,NaN,Small,-31.603,-68.547,5229.0,SAN JUAN AEROCLUB,Argentina,NaN


In [15]:
# Unused attempt at using airports to determine latitude and longitude
# Decided to use google search instead

# airport_array = arg_air['name'].str.lower().str.replace(' airport','')
# arg_loc['airport_index'] = -1

# for row,dat in arg_loc.iterrows():
#     location_text = dat[dat.location_type].lower()
#     location_index = dat.name
    
#     match_value = process.extractOne(location_text, airport_array)[0]
#     match_index = airport_array.loc[airport_array==match_value].index[0]
    
#     arg_loc.loc[location_index, 'airport_index'] = match_index
#     print(location_text, match_value)


In [16]:
save_it(airports, '02_airport_information_fallingrain')